In [ ]:
# Package Imports
import pandas as pd
import os

In [ ]:
# Read data
#le test
books = pd.read_json(r'goodreads_books_young_adult.json', lines=True)

In [ ]:
# View data
books

In [ ]:
# List Columns
list(books.columns)

In [ ]:
# Get rid of some columns (feel free to change)
books = books.drop(columns=['isbn',
                            'text_reviews_count',
                            'series',
                            'country_code',
                            'language_code',
                            'popular_shelves',
                            'asin',
                            'is_ebook',
                            'kindle_asin',
                            'format',
                            'link',
                            'publisher',
                            'publication_day',
                            'isbn13',
                            'publication_month',
                            'edition_information',
                            'publication_year',
                            'url',
                            'image_url'])

In [ ]:
# View new data
books